조엘 그루스, 2016, 인사이트, '밑바닥부터 시작하는 데이터 과학'

In [1]:
from collections import Counter
from functools import partial
import math, random
from linear_algebra import dot

In [2]:
#신경망 기본꼴
def step_function(x):  # 활성함수
    return 1 if x>= 0 else 0
def perceptron_output(weights, bias, x):
    calculation = dot(weights, x) + bias  # 선형분리 y = ax+b 꼴
    return step_function(calculation)

In [3]:
#순방향(Feed-forward) 신경망
def sigmoid(t):  # 활성함수
    return 1 / (1 + math.exp(-t))
def neuron_output(weights, inputs):
    return sigmoid(dot(weights, inputs))     # bias는 weights 리스트에 1을 더한 것으로 포함
def feed_forward(neural_network, input_vector):
    outputs = [] #뉴런의 값을 종합하는 결과리스트
    
    for layer in neural_network:   #한 층을 게산, neural_network는 각 weight를 가진 리스트
        input_with_bias = input_vector + [1] # bias 추가
        output = [neuron_output(neuron, input_with_bias) for neuron in layer] # 한 층에서 각 뉴런의 결과를 계산
        outputs.append(output) # 한 층의 결과를 추가
        
        input_vector = output # 이전 층의 결과가 다음 층의 입력값이 된다.
    
    return outputs

In [4]:
# xor 분리기
xor_network = [ 
               [[20, 20, -30],
               [20, 20, -10]],
               [[-60,60,-30]]
              ]
for x in [0, 1]:
    for y in [0, 1]:
        print(x, y, feed_forward(xor_network, [x, y])[-1])

0 0 [9.38314668300676e-14]
0 1 [0.9999999999999059]
1 0 [0.9999999999999059]
1 1 [9.383146683006828e-14]


backpropagation

In [5]:
def backpropagate(network, input_vector, targets):
    #이전층의 출력과 다음층의 결과값
    hidden_outputs, outputs = feed_forward(network, input_vector)
    #sigmoid함수미분 * 오류(다음층계산값 - 실재결과)
    output_deltas = [output * (1 - output) * (output - target) for output, target in zip(outputs, targets)] 
    #다음층 weight 갱신
    for i, output_neuron in enumerate(network[-1]):
        # i번째 뉴런에 대해
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            # j번째 입력값과 오류값의 차이로 기울기 조금 기울임
            output_neuron[j] -= output_deltas[i] * hidden_output 
    #이전층의 오류값을 뒤로 전파 
    hidden_deltas = [hidden_output * (1 - hidden_output) * dot(output_deltas, [n[i] for n in output_layer]) 
                    for i, hidden_output in enumerate(hidden_outputs)]
    #이전층의 weight 갱신
    for i, hidden_neuron in enumerate(network[0]):
        #i번째 뉴런에 대해
        for j, input in enumerate(input_vector + [1]):
            # j번째 입력값과 오류값의 차이로 기울기 조금 기울임
            hidden_neuron[j] -= hidden_deltas[i] * input

In [6]:
raw_digits = [
          """11111
             1...1
             1...1
             1...1
             11111""",
          """..1..
             ..1..
             ..1..
             ..1..
             ..1..""",
          """11111
             ....1
             11111
             1....
             11111""",
          """11111
             ....1
             11111
             ....1
             11111""",
          """1...1
             1...1
             11111
             ....1
             ....1""",
          """11111
             1....
             11111
             ....1
             11111""",
          """11111
             1....
             11111
             1...1
             11111""",
          """11111
             ....1
             ....1
             ....1
             ....1""",
          """11111
             1...1
             11111
             1...1
             11111""",
          """11111
             1...1
             11111
             ....1
             11111"""]
def make_digit(raw_digit):
    return [1 if c == '1' else 0
            for row in raw_digit.split("\n")
            for c in row.strip()]

In [7]:
import random
random.seed(0)
input_size = 25
num_hidden = 5
output_size = 10
targets = [[1 if i == j else 0 for i in range(10) for j in range(10)]]

In [8]:
def predict(input):
        return feed_forward(network, input)[-1]

inputs = list(map(make_digit, raw_digits))
targets = [[1 if i == j else 0 for i in range(10)] for j in range(10)]

#임의의 weight와 bias 초기화
# 입력층->은닉층의 weight와 bias
hidden_layer = [[random.random() for __ in range(input_size + 1)]
                for __ in range(num_hidden)]
# 은닉층->출력층의 weight와 bias
output_layer = [[random.random() for __ in range(num_hidden + 1)]
                for __ in range(output_size)]
network = [hidden_layer, output_layer]

#1만번 학습시행
for __ in range(10000):
    for input_vector, target_vector in zip(inputs, targets):
        backpropagate(network, input_vector, target_vector)

# 학습 데이터의 결과값 예측하기
for i, input in enumerate(inputs):
    outputs = predict(input)
    print(i, [round(p, 2) for p in outputs])

0 [0.96, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 0.03, 0.0]
1 [0.0, 0.96, 0.03, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2 [0.0, 0.02, 0.96, 0.0, 0.0, 0.03, 0.0, 0.0, 0.0, 0.0]
3 [0.0, 0.03, 0.0, 0.97, 0.0, 0.0, 0.0, 0.02, 0.0, 0.03]
4 [0.0, 0.02, 0.02, 0.0, 0.99, 0.0, 0.0, 0.0, 0.0, 0.0]
5 [0.0, 0.0, 0.02, 0.0, 0.0, 0.96, 0.01, 0.0, 0.02, 0.01]
6 [0.0, 0.0, 0.01, 0.0, 0.01, 0.01, 0.99, 0.0, 0.0, 0.0]
7 [0.02, 0.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.97, 0.0, 0.0]
8 [0.03, 0.0, 0.0, 0.0, 0.0, 0.02, 0.0, 0.0, 0.96, 0.03]
9 [0.0, 0.0, 0.0, 0.01, 0.0, 0.02, 0.0, 0.0, 0.03, 0.95]
